In [1]:
import dlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
from imutils import paths
import tensorflow as tf

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [2]:
def distance(p0, p1):
    return math.sqrt( (p0[0] - p1[0])**2 + (p0[1] - p1[1])**2 )

# Distance between each point in the facial feature
def featureDistance(feature):
    dist = 0
    for i in range(len(feature)-1):
        dist += math.sqrt( ((feature[i][0] - feature[i+1][0])**2)+((feature[i][1] - feature[i+1][1])**2))
        
    return dist

# Area in the facial feature
def featureArea(feature):
    area = 0
    for i in range(len(feature)-1):
        area +=  feature[i][0] * feature[i+1][1] - feature[i][1] * feature[i+1][0]
        
    area /=2
    return abs(area)

def distanceBetweenFeatures(f0, f1):
    f0Mean = np.average(f0, axis=0)
    f1Mean = np.average(f1, axis=0)
    
    return math.sqrt( (f0Mean[0] - f1Mean[0])**2 + (f0Mean[1] - f1Mean[1])**2 )
    

def image_landmarks(image,face_landmarks):
    radius = -1
    circle_thickness = 1
    image_copy = image.copy()
    for (x, y) in face_landmarks:
        cv2.circle(image_copy, (x, y), circle_thickness, (255,0,0), radius)
        plt.imshow(image_copy, interpolation='nearest')
    plt.axis('off')
    plt.show()

In [3]:
# frontalface_detector = dlib.get_frontal_face_detector()
# landmark_predictor=dlib.shape_predictor('Resources\cascade\shape_predictor_68_face_landmarks.dat')
# img = cv2.imread("Resources/images/lena.ppm")

# faces = frontalface_detector(img, 1)
# if len(faces):
#     landmarks = [(p.x, p.y) for p in landmark_predictor(img, faces[0]).parts()]

    
# leftEyeArea = featureArea(np.concatenate((landmarks[36:42], 
#                                           np.array([landmarks[39], landmarks[38], landmarks[37], landmarks[36]]))))

# print(leftEyeArea)

In [7]:
# Facial features index in landmark
# Jawline = 0–16
# Left Eyebrow = 17–21
# Right Eyebrow = 22–26
# Nose = 27–35
# Left eye = 36–41
# Right eye = 42–47
# Mouth = 48–67

# Index for emotions
# 0 - angry
# 1 - disgusted
# 2 - fearful
# 3 - happy
# 4 - neutral
# 5 - sad
# 6 - surprised

frontalface_detector = dlib.get_frontal_face_detector()
landmark_predictor=dlib.shape_predictor('Resources\cascade\shape_predictor_68_face_landmarks.dat')

imagePaths = list(paths.list_images("Resources//data3"))
imagePaths = [imagePath.replace("\\","//",-1) for imagePath in imagePaths]
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    # Extract the class label from directory
    label = imagePath.split("//")[2]
    
    # Convert image to grey scale and 48x48
    img = cv2.imread(imagePath)
    
    faces = frontalface_detector(img, 1)
    if len(faces):
        landmarks = [(p.x, p.y) for p in landmark_predictor(img, faces[0]).parts()]
        eyebrows = featureDistance(landmarks[17:27]) 
        leftEye = featureDistance(landmarks[36:42])
        rightEye = featureDistance(landmarks[42:48])
        mouth = featureDistance(landmarks[48:68])
        
        leftEyeArea = featureArea(np.concatenate((landmarks[17:22], 
                                          np.array([landmarks[39], landmarks[38], landmarks[37], landmarks[36]]))))
        
        rightEyeArea = featureArea(np.concatenate((landmarks[22:27], 
                                          np.array([landmarks[45], landmarks[44], landmarks[43], landmarks[42]]))))
        
        mouthArea = featureArea(landmarks[48:68])
        
        leftEyeAndEyeBrowDist = distanceBetweenFeatures(landmarks[36:42], landmarks[17:22])
        rightEyeAndEyeBrowDist = distanceBetweenFeatures(landmarks[42:48], landmarks[22:27])
        
        distBetweenEyeBrowns = distance(landmarks[21], landmarks[22])
        
        data.append(np.array([eyebrows, leftEye, rightEye, mouth, leftEyeArea, rightEyeArea, mouthArea, 
                             leftEyeAndEyeBrowDist, rightEyeAndEyeBrowDist, distBetweenEyeBrows]))
        labels.append(label)
    

# Convert the data and labels to np arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)    

# Perform one-hot encoding on labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(trainX)
trainX = scaler.transform(trainX)
testX = scaler.transform(testX)

knn = KNeighborsClassifier(n_neighbors=15, p=2, metric='euclidean')

knn.fit(trainX, trainY)

predictions = knn.predict(testX)

print("KNN")
print("Accuracy: " + "{0:.2f}".format(accuracy_score(testY, predictions)*100) + "%")
print(classification_report(testY, predictions))

KNN
Accuracy: 19.66%
              precision    recall  f1-score   support

           0       0.75      0.02      0.05       129
           1       0.00      0.00      0.00        17
           2       0.00      0.00      0.00       129
           3       0.78      0.57      0.66       276
           4       0.39      0.09      0.15       185
           5       0.00      0.00      0.00       135
           6       0.63      0.16      0.25       121

   micro avg       0.70      0.20      0.31       992
   macro avg       0.36      0.12      0.16       992
weighted avg       0.46      0.20      0.25       992
 samples avg       0.20      0.20      0.20       992



C:\Users\Chanys5\anaconda3\envs\tensorflow2\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Chanys5\anaconda3\envs\tensorflow2\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()

dtree.fit(trainX, trainY)

predictions = dtree.predict(testX)

print("Decision Tree")
print("Accuracy: " + "{0:.2f}".format(accuracy_score(testY, predictions)*100) + "%")
print(classification_report(testY, predictions))

Decision Tree
Accuracy: 37.30%
              precision    recall  f1-score   support

           0       0.29      0.27      0.28       129
           1       0.18      0.24      0.21        17
           2       0.21      0.23      0.22       129
           3       0.58      0.60      0.59       276
           4       0.33      0.33      0.33       185
           5       0.20      0.19      0.20       135
           6       0.43      0.40      0.42       121

   micro avg       0.37      0.37      0.37       992
   macro avg       0.32      0.32      0.32       992
weighted avg       0.37      0.37      0.37       992
 samples avg       0.37      0.37      0.37       992



In [6]:
from sklearn.ensemble import RandomForestClassifier

rforest = RandomForestClassifier(criterion="gini", max_depth= 10, max_features="log2",
                                max_leaf_nodes = 100, min_samples_leaf = 3, min_samples_split = 20,
                                n_estimators= 22000, random_state= 5)

rforest.fit(trainX, trainY)

predictions = rforest.predict(testX)

print("Random Forest")
print("Accuracy: " + "{0:.2f}".format(accuracy_score(testY, predictions)*100) + "%")
print(classification_report(testY, predictions))

Random Forest
Accuracy: 16.94%
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       129
           1       0.00      0.00      0.00        17
           2       0.00      0.00      0.00       129
           3       0.76      0.59      0.67       276
           4       0.00      0.00      0.00       185
           5       0.00      0.00      0.00       135
           6       1.00      0.03      0.06       121

   micro avg       0.76      0.17      0.28       992
   macro avg       0.25      0.09      0.10       992
weighted avg       0.33      0.17      0.19       992
 samples avg       0.17      0.17      0.17       992



C:\Users\Chanys5\anaconda3\envs\tensorflow2\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Chanys5\anaconda3\envs\tensorflow2\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
